In [ ]:
import os
import pandas as pd
import numpy as np
import warnings
from sklearn.model_selection import KFold
from sklearn.metrics import roc_auc_score
import lightgbm as lgb
from sklearn.impute import SimpleImputer
import gc
warnings.simplefilter(action='ignore', category=FutureWarning)

In [ ]:
DATA_DIRECTORY = ""

In [ ]:
train = pd.read_csv(os.path.join(DATA_DIRECTORY, 'train.csv'))
test = pd.read_csv(os.path.join(DATA_DIRECTORY, 'test.csv'))
labels = pd.read_csv(os.path.join(DATA_DIRECTORY, 'labels.csv'))

In [ ]:
labels = labels.to_numpy()
test_id = test['SK_ID_CURR']
train_id = train['SK_ID_CURR']

In [ ]:
train = train.drop(['SK_ID_CURR'], axis=1)
test = test.drop(['SK_ID_CURR'], axis=1)

In [ ]:
imputer = SimpleImputer(strategy = 'median')
imputer.fit(train)
train = imputer.transform(train)
test = imputer.transform(test)

In [ ]:
def model(features, test_features, labels, test_ids, n_folds = 5):
    
    
    cat_indices = 'auto'
        
    print('Training Data Shape: ', features.shape)
    print('Testing Data Shape: ', test_features.shape)
    
    
    # Convert to np arrays
    features = np.array(features)
    test_features = np.array(test_features)
    
    # Create the kfold object
    k_fold = KFold(n_splits = n_folds, shuffle = True, random_state = 88)
    
    # Empty array for test predictions
    test_predictions = np.zeros(test_features.shape[0])
    
    # Empty array for out of fold validation predictions
    out_of_fold = np.zeros(features.shape[0])
    
    # Lists for recording validation and training scores
    valid_scores = []
    train_scores = []
    
    # Iterate through each fold
    for train_indices, valid_indices in k_fold.split(features):
        
        # Training data for the fold
        train_features, train_labels = features[train_indices], labels[train_indices]
        # Validation data for the fold
        valid_features, valid_labels = features[valid_indices], labels[valid_indices]
        
        params = {'random_state': 88888, 'nthread': -1}
        # Create the model
        model = lgb.LGBMClassifier(**{**params, **LIGHTGBM_PARAMS})
        
        # Train the model
        model.fit(train_features, train_labels, eval_metric = 'auc',
                  eval_set = [(valid_features, valid_labels), (train_features, train_labels)],
                  eval_names = ['valid', 'train'], categorical_feature = cat_indices,
                  early_stopping_rounds = 2500, verbose = 500)
        
        # Record the best iteration
        best_iteration = model.best_iteration_
        
        # Make predictions
        test_predictions += model.predict_proba(test_features, num_iteration = best_iteration)[:, 1] / k_fold.n_splits
        
        # Record the out of fold predictions
        out_of_fold[valid_indices] = model.predict_proba(valid_features, num_iteration = best_iteration)[:, 1]
        
        # Record the best score
        valid_score = model.best_score_['valid']['auc']
        train_score = model.best_score_['train']['auc']
        
        valid_scores.append(valid_score)
        train_scores.append(train_score)
        
        # Clean up memory
        gc.enable()
        del model, train_features, valid_features
        gc.collect()
        
    # Make the submission dataframe
    submission = pd.DataFrame({'SK_ID_CURR': test_ids, 'TARGET': test_predictions})
    
    # Overall validation score
    valid_auc = roc_auc_score(labels, out_of_fold)
    
    # Add the overall scores to the metrics
    valid_scores.append(valid_auc)
    train_scores.append(np.mean(train_scores))
    
    # Needed for creating dataframe of validation scores
    fold_names = list(range(n_folds))
    fold_names.append('overall')
    
    # Dataframe of validation scores
    metrics = pd.DataFrame({'fold': fold_names,
                            'train': train_scores,
                            'valid': valid_scores}) 
    
    return submission, metrics

In [ ]:
LIGHTGBM_PARAMS = {
    'boosting_type': 'goss',
    'n_estimators': 8888,
    'learning_rate': 0.0098,
    'num_leaves': 58,
    'max_depth': 11,
    'reg_alpha': 3.564,
    'reg_lambda': 4.930,
    'colsample_bytree': 0.613,
    'subsample': 0.708,
    'silent': -1,
    'verbose': -1,
    'max_bin': 407,
    'min_child_weight': 6,
    'min_child_samples': 165
}

In [ ]:
submission, metrics = model(train, train, labels, train_id)

In [ ]:
submission = submission['TARGET'].to_numpy()

In [ ]:
from sklearn.metrics import roc_curve

In [ ]:
fpr, tpr, thresholds = roc_curve(labels, submission)
# Calculate the G-mean
gmean = np.sqrt(tpr * (1 - fpr))

# Find the optimal threshold
index = np.argmax(gmean)
bestThreshold = thresholds[index]

In [ ]:
target = np.where(submission > bestThreshold, 1, 0)

In [ ]:
del fpr, gmean, imputer, index, DATA_DIRECTORY, thresholds, tpr, submission, metrics, LIGHTGBM_PARAMS, bestThreshold
gc.collect()

In [ ]:
from pytorch_tabnet.tab_model import TabNetClassifier
import torch

In [ ]:
def model(features, test_features, labels, test_ids, n_folds = 5):
        
    print('Training Data Shape: ', features.shape)
    print('Testing Data Shape: ', test_features.shape)
    
    # Create the kfold object
    k_fold = KFold(n_splits = n_folds, shuffle = True, random_state = 88)
    
    # Empty array for test predictions
    test_predictions = np.zeros(test_features.shape[0])
    
    # Empty array for out of fold validation predictions
    out_of_fold = np.zeros(features.shape[0])
    
    # Lists for recording validation and training scores
    valid_scores = []
    train_scores = []

    # Iterate through each fold
    for train_indices, valid_indices in k_fold.split(features):
        
        # Training data for the fold
        train_features, train_labels = features[train_indices], labels[train_indices]
        # Validation data for the fold
        valid_features, valid_labels = features[valid_indices], labels[valid_indices]

        # Create the model
        model = TabNetClassifier(
                    n_d=32, 
                    n_a=32, 
                    n_steps=10,
                    gamma=0.098, 
                    n_independent=2, 
                    n_shared=2,
                    lambda_sparse=1e-3, 
                    momentum=0.4, 
                    clip_value=2.,
                    optimizer_fn=torch.optim.Adam,
                    scheduler_params = {"gamma": 0.95,
                                    "step_size": 20},
                    optimizer_params=dict(lr=2e-2),
                    scheduler_fn=torch.optim.lr_scheduler.StepLR, 
                    epsilon=1e-15, verbose = 0,
                    device_name='cuda'
                )
        
        # Train the model
        model.fit(
            train_features, train_labels,
            eval_set=[(train_features, train_labels), (valid_features, valid_labels)],  
            eval_name=['train', 'valid'],
            eval_metric=['auc'],
            max_epochs=1000 , patience=50,
            batch_size=1024, virtual_batch_size=128,
            num_workers=0,
            weights=1,
            drop_last=False
        )

        print(model)
        
        # Make predictions
        test_predictions += model.predict_proba(test_features)[:, 1] / k_fold.n_splits
        
        # Record the out of fold predictions
        out_of_fold[valid_indices] = model.predict_proba(valid_features)[:, 1]

        # Record the best score
        valid_score = roc_auc_score(valid_labels, model.predict(valid_features))
        train_score = roc_auc_score(train_labels, model.predict(train_features))
        
        valid_scores.append(valid_score)
        train_scores.append(train_score)
        
        # Clean up memory
        gc.enable()
        del model, train_features, valid_features
        gc.collect()
        
    # Make the submission dataframe
    submission = pd.DataFrame({'SK_ID_CURR': test_ids, 'TARGET': test_predictions})
    
    # Overall validation score
    valid_auc = roc_auc_score(labels, out_of_fold)
    
    # Add the overall scores to the metrics
    valid_scores.append(valid_auc)
    train_scores.append(np.mean(train_scores))
    
    # Needed for creating dataframe of validation scores
    fold_names = list(range(n_folds))
    fold_names.append('overall')
    
    # Dataframe of validation scores
    metrics = pd.DataFrame({'fold': fold_names,
                            'train': train_scores,
                            'valid': valid_scores}) 
    
    return submission, metrics

In [ ]:
submission, metrics = model(train, test, target, test_id)

In [ ]:
print('TabNet metrics')
print(metrics)

In [ ]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
tn = submission['TARGET'].to_numpy().reshape(-1, 1)
scaler.fit(tn)
tg = scaler.transform(tn)

In [ ]:
submission = pd.DataFrame({'SK_ID_CURR': test_id, 'TARGET': tg[:,0]})

In [ ]:
submission.to_csv('lightgbm-tabnet.csv', index = False)